In [62]:
from typing import Annotated, TypedDict

from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [63]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str

In [64]:
router_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a routing system that determines what type of request a user makes and the requirements needed to complete the request."
            "You have access to the following tools: create_tool, delete_tool." # edit_tool
            # "If the user requests to get, see, or view one or more tasks, you will use get_tool. An example of a user request that meets this criteria is 'What are my tasks'."
            # "If the user requests to modify, change, or edit one or more tasks, you will use the edit tool. An example request that meets this criteria is 'Change my \"clean room\" task to \"clean bathroom\"'."
            "If the user requests to create, make, or add one or more tasks, you will use create_tool. Example requests that meet this critera include: 'Create a \"Clean Room\" task.', 'Create the following tasks: \"Change oil\", \"Go for a run\", and \"Check email\"'."
            "If the user requests to delete, remove, or cancel one or more tasks, you will use the delete_tool. Examples of requests that meet this criteria include 'Delete my \"Change oil\" task.', 'Delete the following tasks: \"Replace lightbuild\" and \"Eat leftovers\"'."
            "If the user requests anything unrelated to their tasks you will NOT use a tool. An example request that meets this criteria is 'Why is the sky blue?'."
        ),
        ("user", "{user_input}")
    ]
)

assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful and concise assistant that aids in the usage of a task management application."
            "You use the information given to you and relay that information to the user in the briefest manner possible."
        ),
        ("placeholder", "{messages}")
    ]
)

In [96]:
tasks = {"Clean Room": {"complete": False}, "Change Oil": {"complete": False}}

In [97]:
# @tool("get_tool", parse_docstring=True)
# def get():
#     """Retrieve the user's tasks."""
#     return tasks

@tool("create_tool", parse_docstring=True)
def create(task_names: list[str]):
    """Create one or more new tasks.

    Args:
        task_names (str): The names of the tasks to create.
    """
    print(f"Tasks being created: {task_names}")
    for task_name in task_names:
        tasks[task_name] = {"complete": False}
        

@tool("delete_tool", parse_docstring=True)
def delete(task_names: list[str]):
    """Delete one or more tasks.

    Args:
        task_names (list[str]): The names of the tasks to delete.
    """
    print(f"Tasks being deleted: {task_names}")
    for task_name in task_names:
        tasks.pop(task_name)

# @tool("edit_tool", parse_docstring=True)
# def edit(task_name: str):
#     """Edit one or more of the user's tasks.

#     Args:
#         task_name (str): The name of the task
#     """

tools = [delete, create]

In [ ]:
router_llm = ChatOllama(
    base_url="http://localhost:11434",
    model="llama3-groq-tool-use:8b",
    temperature=0,
    keep_alive="15m" 
).bind_tools(tools)

assistant_llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:4b-it-qat",
    temperature=0,
    keep_alive="10m" 
)

In [71]:
# if using a routing node (that checks if the request is related to tasks using a SSM) use the Command object either route to the tool-calling agent, or to a responding llm.

In [99]:
def tool_router(state: State):
    if messages := state.get("messages", []):
        ai_message = messages[-1]
        assert isinstance(ai_message, AIMessage), "AIMessage required here."
    else:
        raise ValueError("Messages required here.")
    
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    else:
        return "responder"

In [100]:
def invalid_request(state: State):
    return {"messages": ["Invalid Request"]}

def responder(state: State):
    if messages := state.get("messages", []):
        message = messages[-1]
    else:
        raise ValueError("Messages required here.")
    if isinstance(message, ToolMessage):
        tool_name = message.name
        print(f"The {tool_name} was executed.")
        if tool_name == "create_tool":
            return {"messages": ["One or more tasks were created."]}
        elif tool_name == "get_tool":
            return {"messages": ["One or more tasks were deleted."]}

    else:
        return {"messages": ["Invalid Request."]}

In [101]:
def router_agent(state: State):
    prompt = router_prompt.invoke({"user_input": state["user_input"]})
    return {"messages": [router_llm.invoke(prompt)]}

In [102]:
graph_builder = StateGraph(State)

graph_builder.add_node("router", router_agent)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_node("responder", responder)
#graph_builder.add_node("invalid_request", invalid_request)

graph_builder.add_edge(START, "router")
graph_builder.add_conditional_edges("router", tool_router, {"tools": "tools", "responder": "responder"})
graph_builder.add_edge("tools", "responder")
graph_builder.add_edge("responder", END)

graph = graph_builder.compile()

In [105]:
events = graph.stream({"user_input": "Delete the tasks: 'Go for a run' and 'Change lightbulbs'."}, stream_mode="updates")

for event in events:
    print(event)

{'router': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3-groq-tool-use:8b', 'created_at': '2025-05-30T19:32:13.248703649Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9956385665, 'load_duration': 20281045, 'prompt_eval_count': 412, 'prompt_eval_duration': 1956291485, 'eval_count': 37, 'eval_duration': 7977286580, 'model_name': 'llama3-groq-tool-use:8b'}, id='run--9db5d3a4-314d-481d-b675-5a465107b7c1-0', tool_calls=[{'name': 'delete_tool', 'args': {'task_names': ['Go for a run', 'Change lightbulbs']}, 'id': '245877a4-3c76-4196-b4f0-05d2bde907f5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 412, 'output_tokens': 37, 'total_tokens': 449})]}}
Tasks being deleted: ['Go for a run', 'Change lightbulbs']
{'tools': {'messages': [ToolMessage(content='null', name='delete_tool', id='5a57d4ff-3450-43d0-8318-8d6921d28e77', tool_call_id='245877a4-3c76-4196-b4f0-05d2bde907f5')]}}
The delete_tool was executed.
{'responder': None}


In [106]:
tasks

{'Clean Room': {'complete': False}, 'Change Oil': {'complete': False}}